# Memory

## ConversationBufferMemory

In [4]:
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory()
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input" : "Hi!"}, {"output" : "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

## ConverstionBufferWindowMemory

In [9]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
  return_messages=True,
  k=4
)

def add_message(input, output):
  memory.save_context({"input" : input}, {"output" : output})

add_message("1", "1")

In [10]:
add_message("2", "2")
add_message("3", "3")
add_message("4", "4")

In [13]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

In [12]:
add_message("5", "5")

## ConversationSummaryMemory

In [14]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_77313/1190634598.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.1)
/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_77313/1190634598.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [15]:
add_message("South Kddorea is so pretty", "I wish I could go!!!")

In [16]:
get_history()

{'history': 'Nicolas introduces himself as living in South Korea. The AI responds by expressing admiration for his location, wishing it could visit the country.'}

## ConversationKGMemory

In [18]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [19]:
memory.load_memory_variables({"input": "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [20]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")

In [21]:
memory.load_memory_variables({"inputs": "what does nicolas like"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.', additional_kwargs={}, response_metadata={})]}

## Memory on LLM Chain

In [30]:
# Off the shelf : General perpose chain

from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1
)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  # max_token_limit=80
  max_token_limit=120,
  memory_key="chat_history"
)

template = """

  You are a helpful AI talking a human.

  {chat_history}
  Human: {question}
  Your
"""

chain = LLMChain(
  llm=llm,
  memory=memory,
  # prompt=PromptTemplate.from_template("{question}"),
  prompt=PromptTemplate.from_template(template),
  verbose=True
)

chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:


  You are a helpful AI talking a human.

  
  Human: I live in Seoul
  Your


> Finished chain.


"AI: That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?"

In [32]:
chain.predict(question="My name is Jeseok")



> Entering new LLMChain chain...
Prompt after formatting:


  You are a helpful AI talking a human.

  Human: I live in Seoul
AI: AI: That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?
Human: What is my name?
AI: AI: I don't know your name yet! Would you like to share it with me?
  Human: My name is Jeseok
  Your


> Finished chain.


'AI: Nice to meet you, Jeseok! How do you like living in Seoul?'

In [34]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:


  You are a helpful AI talking a human.

  Human: I live in Seoul
AI: AI: That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?
Human: What is my name?
AI: AI: I don't know your name yet! Would you like to share it with me?
Human: My name is Jeseok
AI: AI: Nice to meet you, Jeseok! How do you like living in Seoul?
  Human: What is my name?
  Your


> Finished chain.


"AI: Your name is Jeseok! It's nice to chat with you. What else would you like to talk about?"

In [33]:
memory.load_memory_variables({})

{'chat_history': "Human: I live in Seoul\nAI: AI: That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?\nHuman: What is my name?\nAI: AI: I don't know your name yet! Would you like to share it with me?\nHuman: My name is Jeseok\nAI: AI: Nice to meet you, Jeseok! How do you like living in Seoul?"}

## Chat Based Memory

In [36]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1
)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  # max_token_limit=80
  max_token_limit=120,
  memory_key="chat_history",
  return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="chat_history"),
  ("human", "{question}")
])

chain = LLMChain(
  llm=llm,
  memory=memory,
  prompt=prompt,
  verbose=True
)

chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: I live in Seoul

> Finished chain.


"That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?"

In [37]:
chain.predict(question="My name is Jeseok")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: I live in Seoul
AI: That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?
Human: My name is Jeseok

> Finished chain.


"Nice to meet you, Jeseok! How's life in Seoul treating you?"

In [38]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: I live in Seoul
AI: That's great! Seoul is a vibrant city with a rich history and a mix of modern and traditional culture. What do you enjoy most about living there?
Human: My name is Jeseok
AI: Nice to meet you, Jeseok! How's life in Seoul treating you?
Human: What is my name?

> Finished chain.


'Your name is Jeseok.'

## LCEL Based Memory

In [44]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1
)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  # max_token_limit=80
  max_token_limit=120,
  memory_key="chat_history",
  return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="chat_history"),
  ("human", "{question}")
])

# def load_memory():
def load_memory(_):
  # print(input)
  return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
  result = chain.invoke({
    "question" : question,
  })
  memory.save_context({"input":question}, {"output":result.content})
  print(result)

# chain.invoke({
#   # "chat_history" : memory.load_memory_variables({})["chat_history"],
#   # "chat_history" : load_memory(),
#   "question" : "My name is Jeseok"
# })

In [45]:
invoke_chain("My name is Jeseok")

content='Nice to meet you, Jeseok! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 26, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKeYW5AEeZTqan07UOatySWLSBgfy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--b7225a2d-890b-4f6d-b70f-ff39b6522ba8-0' usage_metadata={'input_tokens': 26, 'output_tokens': 16, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [46]:
invoke_chain("What is my name?")

content='Your name is Jeseok. How can I help you today, Jeseok?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKeYs4HDBlP3XOt7oEnKE7WxQ8J4L', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--1e4d0146-ad09-4aa0-a871-37e17ba1648d-0' usage_metadata={'input_tokens': 55, 'output_tokens': 18, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
